# Data Reduction and Photometry:

This lab begins the process of reducing the observing project and using photometry techniques to analyze it. 

### 1: Bias and Overscan

__Define the bias. Include both a description of what it looks like physically, why it
occurs, and how we take it into account (how do we remove its effect on our data
images?) Your data is from the ARCTIC instrument - it has two modes (one amplifier read
out or four amplifier read out).__

The bias is the readout of the ccd when it hasn't been exposed to any light. The results of this readout shows that the amplifier has been set to some offset voltage inorder to avoid negative numbers while taking an actual image. What it looks like physically is the entire ccd exposed to light that is very uniform. The noise in a bias scan comes from the electrinics alone. This can be seen below. To take the bias into account

In [ ]:
from astropy.nddata import CCDData
from astropy.visualization import hist
from pathlib import Path

import astropy.io.fits as fits
import glob
import ccdproc as ccdp
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd


In [ ]:
def imagePreview(path, vMin = 5, vMax = 99.7):
    """
    Function that creates plots of all data from fits files at the end of the path passed
    Useful for visual inspection of the data
    """
    fileNames = glob.glob(path)
    openFits = [ fits.open(a) for a in fileNames]

    data = [a[0].data for a in openFits]
    fig, ax = plt.subplots(len(fileNames), 1, figsize= (8,8*len(fileNames)) )

    for i in range(len(fileNames)):
        ax[i].imshow(data[i], vmin=np.percentile(data[i], vMin), vmax=np.percentile(data[i], vMax),
                   origin='lower', cmap='binary_r', interpolation='nearest')

In [ ]:
imagePreview('data/CrabNebula*.fits')

__Why would we use more than one bias frame__

The purpose of combining many bias' is to reduce the read noise in the combined bias.

The first step will be to fix the headers so that they can actually be read in.

In [ ]:
fileNames = glob.glob('data/bias*.fits')
imageData = [fits.open(a) for a in fileNames]

headers = [a[0].header for a in imageData]
for a in headers:
    del a['CD1_1']
    del a['CD1_2']
    del a['CD2_1']
    del a['CD2_2']

In [ ]:
for a in range(len(fileNames)):
    image = imageData[a]
    fits.writeto('data/fixedBias.000{0}.fits'.format(a), image[0].data, headers[a], overwrite = True)

Ok now it will be good to look at the headers. Below I define a function to get the headers and create a pandas dataframe since it's a little easier to look at specific columns using those. I also couldve used the ccdproc built in class `ImageFileCollection`

In [ ]:
def getHeaders(path):
    """
    take the headers and make them into a dataframe so
    that they can be easily examined
    """
    fileNames = glob.glob(path)
    keys = list(imageData[0][0].header.keys())
    dic = {}
    for i in keys:
        dic.update( {i : [a[0].header[i] for a in imageData]} )
    imageInfo = pd.DataFrame(data = dic)
    return imageInfo

In [ ]:
biasHeaders = getHeaders('data/fixedBias*.fits')
biasHeaders[['FILENAME', 'AMPLIST','BSEC11', 'GTGAIN11', 'GTRON11', 'BSEC12',
       'GTGAIN12', 'GTRON12', 'BSEC21', 'GTGAIN21',
       'GTRON21', 'BSEC22', 'GTGAIN22', 'GTRON22']]

Ok Cool, now it looks like these images need to have the overscan trimmed off. That is clear from the columns BSECxx above that show where the overscan regions are. Below I copied and pasted code from the book/website to do that. 

In [ ]:
calibrated_bias = Path('data', 'trimmed_bias')
calibrated_bias.mkdir(exist_ok = True)

obj = CCDData.read(fileNames[0], unit = 'adu') #specific object that is required to use ccdproc